In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install --upgrade ultralytics==8.0.186

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.4/618.4 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstalled

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
#from IPython import display
#display.clear_output()

#import ultralytics
#ultralytics.checks()

In [ ]:
import wandb
from wandb.integration.ultralytics import add_wandb_callback
from ultralytics.models.yolo.model import YOLO
#from ultralytics import YOLO
from IPython.display import display, Image
from roboflow import Roboflow

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



In [ ]:
wandb.login(key='KEY')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

rf = Roboflow(api_key="API_KEY")
project = rf.workspace("alexandr-kolesnikov").project("uml-sd")
dataset = project.version(9).download("yolov8")

/content/datasets
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.186, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to UML-SD-9 in yolov8:: 100%|██████████| 3342/3342 [00:00<00:00, 5979.58it/s]


In [ ]:
#dataset.location
%cd /content/datasets/UML-SD-9
!pwd

/content/datasets/UML-SD-9
/content/datasets/UML-SD-9


In [ ]:
#import gc
#gc.collect()
#wandb.finish()

In [ ]:
#model = YOLO("yolov8m.pt")
#model.train(task='detect', data='data.yaml', epochs=2, imgsz=640, batch=30, optimizer='SGD', plots=True, save=False)

In [ ]:
sweep_config = {
    'project': 'uml-sd',
    'name': 'sweep',
    'method': 'bayes',
    'metric': {
      'name': 'metrics/mAP50(B)',
      'goal': 'maximize'
    },
    'parameters': {
        'model': {
            'values': ['yolov8n', 'yolov8s']
        },
        'epochs': {
          'value': 50
        },
        'optimizer': {
          'values': ['SGD', 'Adam', 'Adamax', 'AdamW', 'NAdam', 'RAdam', 'RMSProp']
        },
        'learning_rate': {
          # a flat distribution between 0 and 0.1
          'distribution': 'uniform',
          'min': 0,
          'max': 0.1
        },
        'batch_size': {
          # integers between 8 and 256
          # with evenly-distributed logarithms
          'distribution': 'q_log_uniform_values',
          'q': 1,
          'min': 8,
          'max': 32,
        }
    }
}

In [ ]:
# Garbage Collector - use it like gc.collect()
import gc

def on_epoch_end(trainer):
    gc.collect()

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Initialize YOLO Model
        model = YOLO(f"{config.model}.pt")
        model.add_callback('on_fit_epoch_end', on_epoch_end)

        # Add W&B callback for Ultralytics
        add_wandb_callback(model, enable_model_checkpointing=True)

        # Train/fine-tune your model
        # At the end of each epoch, predictions on validation batches are logged
        # to a W&B table with insightful and interactive overlays for
        # computer vision tasks
        model.train(task='detect', data='data.yaml', epochs=config.epochs, imgsz=640, batch=config.batch_size, optimizer=config.optimizer, lr0=config.learning_rate, plots=True, save=False, exist_ok=True)
    wandb.finish()

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="uml-sd")

Create sweep with ID: pyduq4hv
Sweep URL: https://wandb.ai/user-xat/uml-sd/sweeps/pyduq4hv


In [ ]:
wandb.agent('pyduq4hv', train, project='uml-sd', count=5)

wandb: Agent Starting Run: 49qkd3gv with config:
wandb: 	batch_size: 25
wandb: 	epochs: 50
wandb: 	learning_rate: 0.019433425607153623
wandb: 	model: yolov8n
wandb: 	optimizer: AdamW
wandb: Currently logged in as: user-xat. Use `wandb login --relogin` to force relogin


Streaming output truncated to the last 5000 lines.
image 1/1 /content/datasets/UML-SD-9/valid/images/image9-27-_png.rf.da1f52ef63eab91c69465cb73628b017.jpg: 384x640 2 actors, 1 label, 1 note, 1 object, 1 package, 2 reply_messages, 2 synchronous_messages, 13.0ms
Speed: 1.4ms preprocess, 13.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/train

image 1/1 /content/datasets/UML-SD-9/valid/images/-3-2-19-_png.rf.73d22ea0439fba568d93e107c9445c6d.jpg: 384x640 1 actor, 2 labels, 2 notes, 1 object, 1 package, 14 recursive_reply_messages, 11 recursive_sync_messages, 3 reply_messages, 4 synchronous_messages, 13.6ms
Speed: 1.3ms preprocess, 13.6ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/train

image 1/1 /content/datasets/UML-SD-9/valid/images/image7-14-_png.rf.95f2ff5a9c4bcad840bffae30effb4b8.jpg: 416x640 1 actor, 4 labels, 1 note, 1 object, 1 package, 1 recursive_reply_message, 3 recursive_sync

lr/pg0,█▅▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▃▆████▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃▆████▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▁▃▅▂▆▇▂▇▇▅▇▇▇▆▆▅▇█▇▇████▆██████▅███████
metrics/mAP50-95(B),▁▁▂▄▁▅▆▂▆▆▄▆▇▇▅▅▅▇▇▇▆▇▇▇▇▅▇▇▇▇▇▇▄▇█▇████
metrics/precision(B),▁▄▄▅▆▇▇▅▇▇▇▇▇▇▆▆▇▇▇▇▇██▇▇▇████▇▇▄█▇▇████
metrics/recall(B),▁▁▃▅▂▆▇▂▆▆▅▇▇▇▅▆▅▇█▇▇▇███▆▇██▇██▅▇██████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁


wandb: Agent Starting Run: c27elfo1 with config:
wandb: 	batch_size: 29
wandb: 	epochs: 50
wandb: 	learning_rate: 0.03099080240725005
wandb: 	model: yolov8n
wandb: 	optimizer: Adam


Выходные данные были обрезаны до нескольких последних строк (5000).
image 1/1 /content/datasets/UML-SD-9/valid/images/-17-3-1-3_jpg.rf.6ed7362f01ec6cd1666336bcf3961f56.jpg: 448x640 2 actors, 1 alternative, 8 labels, 1 loop, 1 note, 1 object, 1 package, 1 reference, 9 reply_messages, 4 synchronous_messages, 17.2ms
Speed: 1.5ms preprocess, 17.2ms inference, 3.4ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/train2

image 1/1 /content/datasets/UML-SD-9/valid/images/-17-3-1_png.rf.9a7316eca2f0557dab874beb6300d49b.jpg: 448x640 2 actors, 4 labels, 1 note, 1 object, 1 optional, 2 packages, 4 reply_messages, 3 synchronous_messages, 16.3ms
Speed: 1.8ms preprocess, 16.3ms inference, 2.5ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/train2

image 1/1 /content/datasets/UML-SD-9/valid/images/-17-3-1-1_jpg.rf.c46bd147cd0649666c9aca5b8b39e5f9.jpg: 448x640 2 actors, 4 labels, 1 note, 1 object, 1 optional, 2 packages, 4 reply_messages, 3 

lr/pg0,█▆▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
lr/pg1,▃▆████▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃▆████▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▂▂▁▂▅▄▃▆▇▅▄▁▆▆▇▅▂▅▆▇▇▅▅▆▇▆▇▇▇▇▇▇███████
metrics/mAP50-95(B),▁▁▂▁▂▄▃▂▅▆▄▃▁▅▆▆▄▂▄▅▆▆▄▄▅▇▅▇▇▇▇▇▇▇▇█████
metrics/precision(B),▁▅▄▂▄▆▅▄▇▆▄▅▃▅▇▇▆▃▅▇█▇▆▅▅█▅▆▇▇▇▇▇▇▇▇████
metrics/recall(B),▁▂▂▁▂▅▄▄▆▇▅▄▁▆▆▇▅▃▅▆▇▇▅▅▆▆▆▇███▇▇▇██████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▅▅▄▄▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁


wandb: Agent Starting Run: rkrejdn7 with config:
wandb: 	batch_size: 9
wandb: 	epochs: 50
wandb: 	learning_rate: 0.07530821178317998
wandb: 	model: yolov8s
wandb: 	optimizer: AdamW


100%|██████████| 21.5M/21.5M [00:00<00:00, 149MB/s]
New https://pypi.org/project/ultralytics/8.0.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.186 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=50, patience=50, batch=9, imgsz=640, save=False, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None,

In [ ]:
model_name = "yolov8n"

# Initialize W&B run
# wandb.init(project="ultralytics", job_type="inference")

# Initialize YOLO Model
model = YOLO(f"{model_name}.pt")

# Add W&B callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

# Train/fine-tune your model
# At the end of each epoch, predictions on validation batches are logged
# to a W&B table with insightful and interactive overlays for
# computer vision tasks
model.train(task='detect', data='data.yaml', epochs=1, imgsz=640, batch=8, optimizer='Adam', plots=False, save=False)

# Finish the W&B run
wandb.finish()

In [ ]:
batches = [-1,16,32,64]
optimizeres =['SGD', 'Adam', 'Adamax', 'AdamW', 'NAdam', 'RAdam', 'RMSProp', 'auto']
epochs = 100

for optimizer in optimizeres:
  for batch in batches:
    model = YOLO('yolov8n.pt')
    results = model.train(task='detect', data='data.yaml', epochs=epochs, imgsz=640, batch=batch, optimizer=optimizer, plots=True, save=True)